# Document Question Answering

An example of using Chroma DB and LangChain to do question answering over documents.

In [2]:
!pip install langchain
!pip install -U langchain-openai
!pip install chromadb
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
OPEN_AI_API_KEY="API_KEY_HERE"
LANGCHAIN_API_KEY="API_KEY_HERE"

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader

## Load documents

Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.

In [5]:
loader = TextLoader('input.txt')
documents = loader.load()

## Split documents

Split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Initialize ChromaDB

Create embeddings for each chunk and insert into the Chroma vector database.

In [9]:
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"] =url = "https://api.langchain.plus/info"
os.environ['OPEN_AI_API_KEY']=OPEN_AI_API_KEY
os.environ['LANGCHAIN_API_KEY']=LANGCHAIN_API_KEY


embeddings = OpenAIEmbeddings(api_key=OPEN_AI_API_KEY)
vectordb = Chroma.from_documents(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Create the chain

Initialize the chain we will use for question answering.

In [12]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(api_key=OPEN_AI_API_KEY), chain_type="stuff", vectorstore=vectordb)

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:289: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


## Ask questions!

Now we can use the chain to ask questions!

In [13]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' The president mentioned that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her qualifications and support from both Democrats and Republicans.'

In [14]:
query = "How are democracies responding to the current global challenges, and what does this suggest about the prevailing global sentiment towards peace and security?"
qa.run(query)

' Democracies are rising to the occasion and showing unity in the face of global challenges, such as the conflict in Ukraine. This suggests that there is a prevailing sentiment towards peace and security, with people around the world, even in Russia, coming together to support Ukraine and reject aggression from autocratic leaders like Putin. This also highlights the importance of alliances, like NATO, in promoting and defending peace and stability in the world.'